In [486]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
# from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [487]:
# Load our csv as a pandas dataframe
btc = pd.read_csv("../../Python/side-projects/btc-price-tracker-enhanced/BTCUSDT-D.csv")

In [488]:
# Create our targets for our dataset
direction = []
for row in range(len(btc)):
    if btc["close"][row] > btc["open"][row]:
        direction.append(0)
    else:
        direction.append(1)

# 0 = long, 1 = short
btc["targets"] = direction

def class_name(label: int) -> str:
    if label == 0:
        return "Long"
    else:
        return "short"

In [489]:
# Function to calculate the difference from one day to the next of a given column of data
def percent_change(dataframe, column_name) -> list:

    variable = dataframe[column_name]
    change = [0]
    
    for row in range(len(dataframe)):

        if row == 0:
            continue

        difference = round((variable[row] - variable[row - 1]) / variable[row-1], 6)
        # change.append(difference)
        if difference > 0:
            change.append(1)
        else:
            change.append(0)
        
    return change

In [490]:
def get_average(dataframe, column, length):
    moving_average = []
    length = length
    
    rolling_average = dataframe[column].rolling(length)
    
    for entry in rolling_average:
        total = 0
        for line in entry:
            total += line
        avg = total / len(entry)
        moving_average.append(round(avg, 6))

    return moving_average

In [491]:
btc["daily_change"] = percent_change(btc, "open")
btc["short_average"] = get_average(btc, "open", 21)
btc["long_average"] = get_average(btc, "open", 50)

In [492]:
price = btc["open"]
trend = [0,0]

for row in range(len(btc)):

    if row == 0 or row == 1:
        continue
    
    if price[row] > price[row-1] > price[row-2] and btc["short_average"][row] > btc["long_average"][row] and price[row] > btc["short_average"][row] and price[row] > btc["short_average"][row]:
        trend.append(1)
    elif price[row] < price[row-1] < price[row-2] and btc["short_average"][row] < btc["long_average"][row] and price[row] < btc["short_average"][row] and price[row] < btc["short_average"][row]:
        trend.append(0.5)
    else:
        trend.append(0)

btc["trend"] = trend

In [493]:
day = []

for row in range(len(btc)):
    day.append(btc["date"][row].split("-")[0])

btc["day"] = day

lb = LabelEncoder()
days = lb.fit_transform(btc["day"])

btc["day"] = days

In [494]:
btc.columns

Index(['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'targets',
       'daily_change', 'short_average', 'long_average', 'trend', 'day'],
      dtype='object')

In [495]:
btc.head()

,date,time,open,high,low,close,volume,targets,daily_change,short_average,long_average,trend,day
0,Thu-23-Dec-21,00:00:00,48610,51432,48030,50794,2246508966,0,0,48610.00,48610.00,0.0,4
1,Fri-24-Dec-21,00:00:00,50794,51980,50402,50785,1837726929,1,1,49702.00,49702.00,0.0,0
2,Sat-25-Dec-21,00:00:00,50785,51149,50100,50406,1147589024,1,0,50063.00,50063.00,0.0,2
3,Sun-26-Dec-21,00:00:00,50406,51294,49283,50763,1676805639,0,0,50148.75,50148.75,0.0,3
4,Mon-27-Dec-21,00:00:00,50763,52119,50464,50706,1765283382,1,1,50271.60,50271.60,0.0,1


In [496]:

features = btc.filter(['targets', "daily_change", "short_average", "long_average", "trend", "day"]) 

X_train_full, X_test, y_train_full, y_test = train_test_split(features.drop(["targets"], axis = 1), features["targets"], random_state=42)

input_number = X_train_full.shape[1]
input_number

5

In [497]:
X_val, X_train = X_train_full[:100], X_train_full[100:]
y_val, y_train = y_train_full[:100], y_train_full[100:]

X_train.shape

(655, 5)

In [498]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(input_number,)),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(2, activation="softmax")
])

model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_181 (Dense)           (None, 500)               3000      
                                                                 
 dense_182 (Dense)           (None, 300)               150300    
                                                                 
 dense_183 (Dense)           (None, 500)               150500    
                                                                 
 dense_184 (Dense)           (None, 2)                 1002      
                                                                 
Total params: 304,802
Trainable params: 304,802
Non-trainable params: 0
_________________________________________________________________


In [499]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])

#### Train

In [500]:
history = model.fit(X_train, y_train, epochs = 500, validation_data=(X_val, y_val))

Epoch 1/500
21/21 [==============================] - 1s 11ms/step - loss: 4291469852742992252633088.0000 - accuracy: 0.4977 - val_loss: 0.6937 - val_accuracy: 0.4700
Epoch 2/500
21/21 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6937 - val_accuracy: 0.4700
Epoch 3/500
21/21 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6937 - val_accuracy: 0.4700
Epoch 4/500
21/21 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6938 - val_accuracy: 0.4700
Epoch 5/500
21/21 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6938 - val_accuracy: 0.4700
Epoch 6/500
21/21 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6938 - val_accuracy: 0.4700
Epoch 7/500
21/21 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6938 - val_accur

#### Plot

In [ ]:
pd.DataFrame(history.history).plot(figsize=(20, 10))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [346]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5198


[0.6921373009681702, 0.5198412537574768]

In [173]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

1/1 [==============================] - 0s 72ms/step


array([[0.47, 0.53],
       [0.43, 0.57],
       [0.5 , 0.5 ]], dtype=float32)

In [160]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

1/1 [==============================] - 0s 24ms/step


array([0, 1, 0])

In [161]:
print(class_name(y_pred[0]), class_name(y_pred[1]), class_name(y_pred[2]))

Long short Long


In [162]:
y_new = y_test[:3]
y_new

926    0
630    0
682    1
Name: targets, dtype: int64